In [5]:
import requests
import math
import pandas as pd
import urllib

# Saisie manuelle de l'adresse et du radius

In [ ]:
adresse = input("Veuillez saisir une adresse comlète.ex: 10 rue de Paris Paris 75001 \n").replace(" ","+")
lat, lon = getCoordinates(adresse)
radius_km = int(input("Saisir la portée en km"))
resultat = requete(adresse,radius_km)
resultat

### Exemple Liste des stations de vélos 10 km autour de la mairie de Lyon

In [412]:
adresse = "19 Quai Tilsitt, 69002 Lyon"
radius_km = 1
resultat = requete(adresse,radius_km)
resultat

                                    name    place ville                  lat  \
0                      Perrache / Carnot  station  Lyon  45.75009964332607e0   
1                           Place Regaud  station  Lyon    45.763076929567e0   
2                   Place Antonin Gourju  station  Lyon   45.7594863490351e0   
3                             Saint-Just  station  Lyon  45.75596664224508e0   
4                       Remparts d'Ainay  station  Lyon  45.75308920956422e0   
5                     Joffre / Vaubecour  station  Lyon  45.75184824275554e0   
6                   Gryphe / Montesquieu  station  Lyon  45.75191143931073e0   
7                   Place Antonin Poncet  station  Lyon  45.75676232130322e0   
8                           Place Carnot  station  Lyon  45.75122705218227e0   
9                         Place Guichard  station  Lyon  45.75881520504663e0   
10                         Sainte-Irénée  station  Lyon    45.754945881074e0   
11                  Childebert / Rivière

### Exemple Liste des stations de vélos 10 km autour de la mairie de Rennes

In [406]:
adresse = "17 Bis Rue de la Monnaie, 35000 Rennes"
radius_km = 10
resultat = requete(adresse,radius_km)
resultat

                           name    place   ville          lat          lon
0              Musée Beaux-Arts  station  Rennes  48.109601e0   -1.67408e0
1                 Champs Libres  station  Rennes  48.105537e0  -1.674328e0
2                Brest - Verdun  station  Rennes  48.113009e0  -1.693264e0
3                       Marbeuf  station  Rennes  48.111749e0  -1.702077e0
4                Plaine de Baud  station  Rennes  48.112461e0  -1.643674e0
5                    Gros-Chêne  station  Rennes  48.126795e0  -1.665101e0
6                      Painlevé  station  Rennes  48.123506e0  -1.659762e0
7  Joliot-Curie - Chateaubriand  station  Rennes  48.124389e0  -1.651203e0
8                    La Poterie  station  Rennes  48.087282e0  -1.644121e0
9            Pont de Strasbourg  station  Rennes  48.109859e0  -1.656028e0


### Exemple Liste des stations de vélos 10 km autour de la mairie de Lyon

In [407]:
adresse = "9 Rue Lavoisier Lille 59800 Lille"
radius_km = 10
resultat = requete(adresse,radius_km)
resultat

                 name     place  ville              lat              lon
0               LILLE  hospital  Lille  50.6465113784e0  3.08395241534e0
1               LILLE  hospital  Lille  50.6202380916e0  3.07717927762e0
2               LILLE  hospital  Lille  50.6339108151e0  3.03153623994e0
3                LOOS  hospital  Lille  50.6105672954e0  3.00789073236e0
4   VILLENEUVE D'ASCQ  hospital  Lille  50.6530104252e0  3.15218579198e0
5       (LOMME) LILLE  hospital  Lille  50.6513947118e0   2.9731517883e0
6          HAUBOURDIN  hospital  Lille  50.6130128507e0  2.98673462341e0
7              SECLIN  hospital  Lille  50.5466414558e0  3.01781005107e0
8             LESQUIN  hospital  Lille  50.5888971553e0  3.08948682714e0
9    MARCQ-EN-BAROEUL  hospital  Lille   50.666496025e0  3.10155008906e0
10      (LOMME) LILLE  hospital  Lille   50.653891345e0   2.9923404297e0
11              LILLE  hospital  Lille  50.6491473003e0  3.08549912212e0
12              LILLE  hospital  Lille  50.60866967

In [403]:
def getCoordinates(adresse):
    adresse = adresse.replace(" ","+")
    requete = "https://api-adresse.data.gouv.fr/search/?q="+adresse
    response = requests.get(requete)
    json = response.json()
    lat = json["features"][0]["geometry"]["coordinates"][1]
    lon = json["features"][0]["geometry"]["coordinates"][0]
    return lat,lon

In [ ]:
def convRadiusKM(radius_km):
    return math.pow(radius_km/111.1949,2)

In [404]:
def requete(adresse,radius):
    lat, lon = getCoordinates(adresse)
    radius_degree = convRadiusKM(radius_km)
    phi = round(lat)
    val1 = math.cos(phi)*math.sin(phi)* math.pi/180
    val2 = math.cos(phi)*(math.cos(phi) - math.sin(phi)* math.pi/180 * (lat-2*phi))
    
    # Lyon Velos
    PREFIX = "PREFIX ns: <http://schema.org/> "\
        + "PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> "\
        + "PREFIX rdfs:<http://www.w3.org/2001/XMLSchema#> "\
        + "PREFIX owl: <http://www.owl-ontologies.com/unnamed.owl#> "
    
    FILTER = "FILTER (("+str(lat)+"-?lat)*("+str(lat)+"-?lat) + ("+str(lon)+"-?long)*("+str(lon)+"-?long)* " \
    "("+str(val2)+"-("+str(val1)+"*?lat)) < "+str(radius_degree)+")}"
    
    requete = "SELECT ?ville ?name ?available_bikes ?available_bikes_stands ?bike_stands ?lat ?long ?place WHERE { " \
    "?station rdf:type ns:station . ?station ns:name ?name . ?station ns:ville ?ville . ?station ns:available_bikes ?available_bikes . "  \
    "?station ns:available_bikes_stands ?available_bikes_stands . ?station ns:long ?long . ?station ns:lat ?lat . ?station ns:place ?place . "
    sparql_query1 = PREFIX + requete + FILTER
    #print(sparql_query1)
    sparql_query1 = urllib.parse.quote(sparql_query1, safe='')
    url1 = "http://localhost:3030/db/sparql?query="+sparql_query1
    #print(url1)
    response1 = requests.get(url1)
    json1 = response1.json()
    json1 = json1["results"]["bindings"]
    
    requete1 = pd.DataFrame()
    
    for i in range(len(json1)):
        name = json1[i]["name"]["value"] 
        ville = json1[i]["ville"]["value"]
        lat = json1[i]["lat"]["value"]
        lon = json1[i]["long"]["value"]
        place = json1[i]["place"]["value"]
        list_to_append = [name] + [ville] + [lat] + [lon] + [place]
        line_labels = ["name"] + ["ville"] + ["lat"] + ["lon"] + ["place"]
        dic1 = {line_labels[i]: list_to_append[i] for i in range(len(list_to_append))}
        requete1 = requete1.append(dic1,ignore_index=True)
    
    if len(requete1)>1:
        requete1 = requete1[["name","place","ville","lat","lon"]]
        print(requete1)
    
    
    # Lille Hopitaux
    PREFIX = "PREFIX ns: <http://schema.org/> " \
            "PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> "\
            "PREFIX rdfs:<http://www.w3.org/2001/XMLSchema#> "\
            "PREFIX owl: <http://www.owl-ontologies.com/unnamed.owl#> "
    
    requete = "SELECT ?commune ?place ?ville ?lat ?long " \
                "WHERE { ?hospital rdf:type ns:hospital . "\
                "?hospital ns:commune ?commune . "\
                "?hospital ns:ville ?ville . "\
                "?hospital ns:long ?long . "\
                "?hospital ns:lat ?lat . "\
                "?hospital ns:place ?place ."
    
    sparql_query2 = PREFIX + requete + FILTER
    #print(sparql_query2)
    sparql_query2 = urllib.parse.quote(sparql_query2, safe='')
    url2 = "http://localhost:3030/db/sparql?query="+sparql_query2
    response2 = requests.get(url2)
    json2 = response2.json()
    json2 = json2["results"]["bindings"]
    #print(json2)
    requete2 = pd.DataFrame()
    
    for i in range(len(json2)):
        name = json2[i]["commune"]["value"] 
        ville = json2[i]["ville"]["value"]
        lat = json2[i]["lat"]["value"]
        lon = json2[i]["long"]["value"]
        place = json2[i]["place"]["value"]
        list_to_append = [name] + [ville] + [lat] + [lon] + [place]
        line_labels = ["name"] + ["ville"] + ["lat"] + ["lon"] + ["place"]
        dic = {line_labels[i]: list_to_append[i] for i in range(len(list_to_append))}
        requete2 = requete2.append(dic,ignore_index=True)
    
    if len(requete2)>1:
        requete2 = requete2[["name","place","ville","lat","lon"]]
        print(requete2)
    
    # Rennes Velos
    PREFIX = "PREFIX ns: <http://schema.org/> "\
    + "PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> "\
    + "PREFIX rdfs:<http://www.w3.org/2001/XMLSchema#> "\
    + "PREFIX owl: <http://www.owl-ontologies.com/unnamed.owl#> "
    
    requete = "SELECT ?nom ?place ?ville ?lat ?long "\
    "WHERE { ?station rdf:type ns:station . ?station ns:nom ?nom ."\
    " ?station ns:ville ?ville . ?station ns:long ?long ."\
    " ?station ns:lat ?lat ." \
    " ?station ns:place ?place .  "
    
    sparql_query3 = PREFIX + requete + FILTER
    #print(sparql_query3)
    sparql_query3 = urllib.parse.quote(sparql_query3, safe='')
    url3 = "http://localhost:3030/db/sparql?query="+sparql_query3
    response3 = requests.get(url3)
    json3 = response3.json()
    json3 = json3["results"]["bindings"]
    
    requete3 = pd.DataFrame()
    
    for i in range(len(json3)):
        name = json3[i]["nom"]["value"] 
        ville = json3[i]["ville"]["value"]
        lat = json3[i]["lat"]["value"]
        lon = json3[i]["long"]["value"]
        place = json3[i]["place"]["value"]
        list_to_append = [name] + [ville] + [lat] + [lon] + [place]
        line_labels = ["name"] + ["ville"] + ["lat"] + ["lon"] + ["place"]
        dic3 = {line_labels[i]: list_to_append[i] for i in range(len(list_to_append))}
        requete3 = requete3.append(dic3,ignore_index=True)
    
    if len(requete3) >1:
        requete3 = requete3[["name","place","ville","lat","lon"]]
        print(requete3)
    